In [25]:
# to import different librariries !{sys.executable} -m pip install
import pandas as pd
import numpy as np

df_0 = pd.read_excel("/Users/jadmoawad/Desktop/Article 7 Experiment_redistribution/Data/part_0.xlsx", skiprows=[1])
df_0['part'] = "pilot"

df_1 = pd.read_excel("/Users/jadmoawad/Desktop/Article 7 Experiment_redistribution/Data/part_1.xlsx", skiprows=[1])
df_1['part'] = "part 1"

df_2 = pd.read_excel("/Users/jadmoawad/Desktop/Article 7 Experiment_redistribution/Data/part_2.xlsx", skiprows=[1])
df_2['part'] = "part 2"

df_3 = pd.read_excel("/Users/jadmoawad/Desktop/Article 7 Experiment_redistribution/Data/part_3.xlsx", skiprows=[1])
df_3['part'] = "part 3"


df = pd.concat([df_0, df_1, df_2, df_3], ignore_index=True)


# Drop respondents who failed the attention check
# After attention check filtering
df = df[(~df['attention_check_1'].isna()) & (df['attention_check_1'] == 1) & 
        (~df['attention_check_3'].isna()) & (df['attention_check_3'] == 1)].copy()

# Then the recaptcha code runs without warning
df['Q_RecaptchaScore'] = pd.to_numeric(df['Q_RecaptchaScore'])
recaptcha_fail = df[df['Q_RecaptchaScore'] < 0.5]
df = df[df['Q_RecaptchaScore'] >= 0.5]

# Harmonize variables  ---------------------------------------------------------
# Initial investment
df['stock'] = df[['O1_1', 'O2_1', 'O3_1', 'O4_1', 'O5_1', 'O6_1', 'O7_1', 'O8_1', 'O9_1', 'O10_1']].bfill(axis=1).iloc[:, 0]
df['etf'] = df[['O1_2', 'O2_2', 'O3_2', 'O4_2', 'O5_2', 'O6_2', 'O7_2', 'O8_2', 'O9_2', 'O10_2']].bfill(axis=1).iloc[:, 0]
df['saving'] = df[['O1_3', 'O2_3', 'O3_3', 'O4_3', 'O5_3', 'O6_3', 'O7_3', 'O8_3', 'O9_3', 'O10_3']].bfill(axis=1).iloc[:, 0]
df['gold'] = df[['O1_4', 'O2_4', 'O3_4', 'O4_4', 'O5_4', 'O6_4', 'O7_4', 'O8_4', 'O9_4', 'O10_4']].bfill(axis=1).iloc[:, 0]
df['crypto'] = df[['O1_5', 'O2_5', 'O3_5', 'O4_5', 'O5_5', 'O6_5', 'O7_5', 'O8_5', 'O9_5', 'O10_5']].bfill(axis=1).iloc[:, 0]

# Lottery allocation
df = df.rename(columns={'lottery_4': 'lot_stock', 'lottery_3': 'lot_etf', 'lottery_5': 'lot_gold', 
                        'lottery_2': 'lot_saving', 'lottery_6': 'lot_crypto'})

# Convert all specified columns to numeric
numeric_columns = ["stock", "etf", "saving", "gold", "crypto", 
                   "lot_stock", "lot_etf", "lot_saving", "lot_gold", "lot_crypto",
                   "age", "sub_financial_know", "risk_tolerance", 
                   "political_placement", "trust_government"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)


# First ensure p_income is numeric
df['p_income'] = pd.to_numeric(df['p_income'], errors='coerce')

# Create income variable with proper NaN handling
df['income'] = np.select(
    [df['p_income'] == -1, 
     df['p_income'].isna(),
     (df['p_income'] >= 1) & (df['p_income'] <= 4),
     df['p_income'] > 4],
    [np.nan, np.nan, 'Below median', 'Above median'],
    default=np.nan
)
df['income'] = pd.Categorical(df['income'], categories=["Below median", "Above median"])


# Education 
df['education_r'] = pd.to_numeric(df['education_r'], errors='coerce')
df['education'] = np.where(df['education_r'] == 7, np.nan,
                          np.where((df['education_r'] >= 1) & (df['education_r'] <= 4), 0, 1))
df['education'] = pd.Categorical(df['education'], categories=[0, 1]).rename_categories(["Some college or lower", "Bachelor's or higher"])


# Race 
race_columns = ["race_1", "race_2", "race_3", "race_4", "race_5", "race_6"]

# Fill NaN with 0 for calculation purposes
for col in race_columns:
    df[col] = df[col].fillna(0)

# Create race variable
df['race'] = pd.Series(dtype='object') 
df.loc[(df['race_1'] == 1) & (df[["race_2", "race_3", "race_4", "race_5", "race_6"]].sum(axis=1) == 0), 'race'] = "White"
df.loc[(df['race_2'] == 1) & (df[["race_1", "race_3", "race_4", "race_5", "race_6"]].sum(axis=1) == 0), 'race'] = "Black"
df.loc[df['race'].isna(), 'race'] = "Other"
df['race'] = pd.Categorical(df['race'], categories=["White", "Black", "Other"])


# Financial literacy
df['financial_literacy'] = pd.Categorical(
    np.where(df['financial_literacy_1'] == 3.0, "Correct", "Incorrect"),
    categories=["Correct", "Incorrect"])


# Treatment group
df['treatment'] = df['treatment_group'].fillna(0)


# Gender
df['gender'] = pd.Categorical(
    np.where(df['sex'] == 1.0, "Men", 
             np.where(df['sex'] == 2.0, "Women", np.nan)),
    categories=["Men", "Women"])


# Relationship status 
df['relationship_status'] = pd.Categorical(
    np.where(df['relationship_status'] == 2.0, 
             "Married/ Cohabited/ In a Union",
             "Single/ other"))


# Convert to numeric / 1= both parents US born
df['parents_us_born'] = pd.Categorical(
    np.where(df['parents_us_born'] == 1.0, "Yes", 
             np.where(df['parents_us_born'] == 2.0, "No", np.nan)),
    categories=["Yes", "No"])

# Replace -1 with NA in both columns
df.loc[df['education_m'] == -1.0, 'education_m'] = np.nan
df.loc[df['education_f'] == -1.0, 'education_f'] = np.nan

# Take the highest among both parents
df['social_origin'] = pd.Categorical(
    np.where((df['education_m'].isin([5.0, 6.0])) | (df['education_f'].isin([5.0, 6.0])),
             "High", "Medium/Low"),
    categories=["High", "Medium/Low"])


# Labor force participation
df['labor_force'] = pd.Categorical(
    np.where(df['status_r'].isin([1.0, 2.0, 3.0, 4.0]),
             "Active", "Not active"),
    categories=["Active", "Not active"])


# House owner - missing closing parenthesis fixed too
df['house'] = pd.Categorical(
    np.where(df['house_status'].isin([1.0, 2.0]),
             "Owner", "Rent/other"),
    categories=["Owner", "Rent/other"])

df = df[["stock", "etf", "saving", "gold", "crypto", 
         "lot_stock", "lot_etf", "lot_saving", "lot_gold", "lot_crypto",
         "age", "sub_financial_know", "risk_tolerance", 
         "political_placement", "trust_government", "house", "labor_force",
         "social_origin", "parents_us_born", "relationship_status", "gender", "treatment", 
         "financial_literacy", "race", "education", "income"]]

before_na_removal = df.copy()
df = df.dropna()

# Save the processed data
df.to_pickle('/Users/jadmoawad/Desktop/Github_financial_experiment/financial-literacy-experiment/data/processed_df.pkl')

/Users/jadmoawad/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/jadmoawad/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/jadmoawad/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/jadmoawad/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
